# Module 5

## Video 24: Filtering by Product
**Python for the Energy Industry**

We can filter Cargo Movements by the carried product in much the same way as we filter by geography. 

## Products Endpoint

[Products endpoint documentation](https://vortechsa.github.io/python-sdk/endpoints/products/)

We start by doing a search for 'crude':

In [1]:
# Some initial imports / settings
from datetime import datetime

from dateutil.relativedelta import relativedelta

import vortexasdk as v

now = datetime.utcnow()
one_month_ago = now - relativedelta(months=1)

In [2]:
crude_search = v.Products().search(term=['crude'])

2020-12-10 16:47:54,856 vortexasdk.client — WARNING — You are using vortexasdk version 0.26.0, however version 0.27.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


In [3]:
len(crude_search)

19

We get 19 results. Inspecting the first of these, we can see there is a product name and ID, plus information about parent products.

In [4]:
crude_search[0]

{'id': 'e1b3d075a9340969322024a35b8c720e5065d8c95a91b6d2cda2cb06dd707717',
 'name': 'Crude Vegoil',
 'layer': ['grade'],
 'leaf': True,
 'parent': [{'name': 'Biodiesel Feedstock',
   'layer': ['category'],
   'id': '9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e489734522f9cdf804c'}],
 'meta': {},
 'ref_type': 'product',
 'hierarchy': [{'id': 'b68cbb746f8b9098c50e2ba36bcad83001a53bd362e9031fb49085d02c36659c',
   'layer': 'group',
   'label': 'Clean Petroleum Products'},
  {'id': 'a75fcc09bfc7d16496de3336551bc52b5891838bb7c22356d2cb65451587d1e5',
   'layer': 'group_product',
   'label': 'Biodiesel'},
  {'id': '9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e489734522f9cdf804c',
   'layer': 'category',
   'label': 'Biodiesel Feedstock'},
  {'id': 'e1b3d075a9340969322024a35b8c720e5065d8c95a91b6d2cda2cb06dd707717',
   'layer': 'grade',
   'label': 'Crude Vegoil'}]}

Lets take a look at all the products returned by our search:

In [5]:
crude_search.to_df()

,id,name,layer.0,parent.0.name
0,e1b3d075a9340969322024a35b8c720e5065d8c95a91b6...,Crude Vegoil,grade,Biodiesel Feedstock
1,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,Crude,group_product,Crude/Condensates
2,4fe046d7478c4072b9ecbaa2c03d3b977bbfec5077ee44...,Crude Butadiene,grade,Olefins/Other Chemicals
3,54af755a090118dcf9b0724c9a4e9f14745c26165385ff...,Crude/Condensates,group,NaN
4,9fda040ee8844e47b5239051e322d06dd9d2b96f0c3249...,TPAO crude,grade,Medium-Sour
5,2b76490350005604507ca64567101df7d3db80a973f462...,Crude Vegetable Oil,grade,Biodiesel Feedstock
6,f26cd12252b0bb23b4ab4ab590eaee11037d78a7ac5cc6...,Crude Benzene,grade,Chemicals
7,e9d1031a9167fff80ef089f2d5591deb1833ee34f1d028...,Crude Palm Oil,grade,Biodiesel Feedstock
8,c2aa8895d3d176868cbb0519f2bb5318a74a2a66ac9034...,Eagle Ford crude,grade,Light-Sweet
9,164a4a510343458bbf949a79964ce3bb8efa8d14276c48...,US Shale Crude,grade,Light-Sweet


There are different layers of product type, ranging from the broad group 'Crude/Condensates', down to specific grades. We can grab the ID of the Crude group/product to filter on.

## Filtering by Products

An assert statement will check that we only get one result for our exact term match.

In [6]:
# Grab the ID for Crude
crude_search = v.Products().search(term=['Crude'],exact_term_match=True)
assert len(crude_search) == 1
crude_ID = crude_search[0]['id']

cm_crude_query = v.CargoMovements().search(
    filter_activity="loading_state",
    filter_time_min=one_month_ago,
    filter_time_max=now,
    filter_products=crude_ID)

Loading from API: 2500it [00:01, 1318.26it/s]             


As before, we pick the colulmns of interest from our Cargo Movements search and make a DataFrame.

In [7]:
required_columns = ["vessels.0.name","vessels.0.vessel_class","product.group.label","product.grade.label","quantity",
                    "status","events.cargo_port_load_event.0.location.port.label","events.cargo_port_load_event.0.end_timestamp",
                    "events.cargo_port_unload_event.0.location.port.label","events.cargo_port_unload_event.0.end_timestamp"]

new_labels = ["vessel_name","vessel_class","product_group","product_grade","quantity","status","loading_port","loading_finish","unloading_port","unloading_finish"]

relabel = dict(zip(required_columns,new_labels))

In [8]:
cm_crude_query.to_df(columns=required_columns).rename(relabel,axis=1)

,vessel_name,vessel_class,product_group,product_grade,quantity,status,loading_port,loading_finish,unloading_port,unloading_finish
0,ASAHI PRINCESS,aframax,Crude/Condensates,NaN,190455,unloading_state,Malampaya [PH],2020-12-05 05:59:44+00:00,Map Ta Phut [TH],NaT
1,KALAHARI,aframax,Crude/Condensates,Siberian Light,585442,unloaded_state,Novorossiysk [RU],2020-11-21 10:52:20+00:00,Burgas (Bourgas) [BG],2020-11-24 19:37:04+00:00
2,M T VEDMATA,tiny_tanker,Crude/Condensates,NaN,45280,unloaded_state,Agge [NG],2020-11-17 04:38:46+00:00,Tulja [NG],2020-11-17 15:15:33+00:00
3,SHALAMAR,aframax,Crude/Condensates,Arab Light,730874,unloaded_state,Ras Tanura [SA],2020-12-01 01:30:06+00:00,"Karachi, SD [PK]",2020-12-08 22:52:33+00:00
4,NEW ENTERPRISE,vlcc_plus,Crude/Condensates,Murban,1132667,transiting_state,Fujairah [AE],2020-11-21 18:35:28+00:00,Tomakomai [JP],NaT
...,...,...,...,...,...,...,...,...,...,...
2002,AMAZON FALCON,panamax,Crude/Condensates,Vasconia,476332,unloaded_state,Covenas [CO],2020-11-16 17:05:04+00:00,Chiriqui Grande [PA],2020-11-19 23:07:19+00:00
2003,A SYMPHONY,suezmax,Crude/Condensates,NaN,1021406,storing_state,NaN,2020-12-08 19:54:29+00:00,NaN,NaT
2004,VAIL SPIRIT,suezmax,Crude/Condensates,Bonny Light,1004770,unloaded_state,Bonny Offshore Oil Terminal [NG],2020-11-15 03:38:20+00:00,Jose Ignacio Terminal [UY],2020-12-03 00:20:30+00:00
2005,DAKOTA STRENGHT,aframax,Crude/Condensates,Hibernia,401320,unloading_state,Whiffen Head [CA],2020-12-05 12:27:34+00:00,"New York, NY [US]",NaT


## Exploring the Products dataset

Let's say you wanted to know which child products belonged to a particular group or category. One way to do this is to full the full products dataset:

In [9]:
search = v.Products().search()

Loading from API: 2000it [00:00, 5183.91it/s]             


In [10]:
len(search)

1762

We can convert this into a DataFrame. This allows us to, for example, find all products of 'group' type.

In [11]:
search_df = search.to_df()

search_df[search_df['layer.0'] == 'group']

,id,name,layer.0,parent.0.name
1155,b68cbb746f8b9098c50e2ba36bcad83001a53bd362e903...,Clean Petroleum Products,group,NaN
1198,54af755a090118dcf9b0724c9a4e9f14745c26165385ff...,Crude/Condensates,group,NaN
1290,5de0b00094e0fd7542c10f9f8a71b4008d55750f21dc90...,Dirty Petroleum Products,group,NaN


Let's say we then want to find the 'child' products of 'Crude/Condensates':

In [12]:
search_df[search_df['parent.0.name'] == 'Crude/Condensates']

,id,name,layer.0,parent.0.name
1170,d053227a2df2f4d6eacca61341cd6c08e49ba6970cc18a...,Condensates,group_product,Crude/Condensates
1188,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,Crude,group_product,Crude/Condensates


Going one step further:

In [13]:
search_df[search_df['parent.0.name'] == 'Crude']

,id,name,layer.0,parent.0.name
64,a7e26956fbb91d786b1d55582981b0d199f72c050958fd...,Medium-Sour,category,Crude
65,1c10c409e5412d5eee38bbbd529509d2fe0251fa99e292...,Medium-Sweet,category,Crude
1552,fc2e0746428ed6109045fc1195a4ee4ef2e339d55b7b6e...,Heavy-Sour,category,Crude
1553,3cb81242a4b490d97b78657e8f1d40b994c32d4334051f...,Heavy-Sweet,category,Crude
1735,87acdcdd0f1b2482eb808d5ea2afdc05266f27e4c9b0a5...,Light-Sour,category,Crude
1736,66a98cd9df06660555000f8d4de9bc075e651bce3b57a4...,Light-Sweet,category,Crude


And finally, to grade level:

In [14]:
search_df[search_df['parent.0.name'] == 'Heavy-Sweet']

,id,name,layer.0,parent.0.name
54,fb4d9f3e8c547b7b5be2f69f72279712a11de9deedec42...,Mayna,grade,Heavy-Sweet
158,f771796213919845444b4310aa8d6184cc8437b2d29de9...,Nigerian Medium,grade,Heavy-Sweet
199,8be6e28c0b2379e8c74cfea67af4dd570e5ce3dbbb3893...,Olende,grade,Heavy-Sweet
201,3a2b4996b8568a15296f541dc0c16d6060e32f2eb903b5...,Olombendo,grade,Heavy-Sweet
214,382a1fdb7c456190f77ed096a3fcc61ff3f77e5d8fd398...,Ostra,grade,Heavy-Sweet
266,2da055a25689552c9a58bba5a3294114aba62cf73e3bde...,Pazflor,grade,Heavy-Sweet
270,acccc718f28b757b433b8809892bc7f4a2d7a7d3e83ad3...,Penglai,grade,Heavy-Sweet
338,497768f75d5f0834a88b6e51a8f805324f3e23b4d2caf6...,Pyrenees,grade,Heavy-Sweet
347,5bcfb3d6ea2b417593fc2524366fbcd74571d6c1bbda4f...,Qinghuangdao,grade,Heavy-Sweet
419,19c99edb8bfa5ecd4efbf3dc4e0017ce338c7a62b49c23...,Roncador Heavy,grade,Heavy-Sweet


This is a convenient way of exploring the different products that exist in the data. 

### Exercise

Assemble a DataFrame of Cargo Movements that were loading in the last 7 weeks, and that were carrying Diesel out of the United States.

*Note: Diesel is a subset of wider group Diesel/Gasoil.*